In [ ]:
"""
Get data of google search results of books and authors to create book/author poularity.

functions used:
book_google(book,author,date) from fictiondb.py
book_google2(book,author,date) from fictiondb.py (improve search results)

Output: book_history_2 (search results that make more sense)
"""

In [2]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import numpy as np
import pickle

import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *

In [2]:
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [3]:
%run -i '../py/fictiondb.py'

In [4]:
driver.get("https://www.google.com")

In [11]:
# Load combined_df to get author,book,release_date of movie
combined_df = pd.read_pickle('../dump/combined_data')
essential_df = combined_df[['movie_title','author']].copy()
essential_df['date'] = combined_df.release_date.dt.strftime('%Y-%m-%d')
essential_df = essential_df.drop_duplicates(subset=['movie_title','date']).copy()
essential_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199 entries, 0 to 1241
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_title  1199 non-null   object
 1   author       1199 non-null   object
 2   date         1199 non-null   object
dtypes: object(3)
memory usage: 37.5+ KB


In [13]:
# Create list to for searching
title_list = list(essential_df.movie_title)
author_list = list(essential_df.author)
date_list = list(essential_df.date)
lng = len(title_list)

In [154]:
%run -i '../py/fictiondb.py'

In [ ]:
book_history = []

In [130]:
for i in range(lng):
    book = title_list[i]
    author = author_list[i]
    date = date_list[i]
    
    print(i,book) #keep track the scraping progress
    
    info = book_google(book,author,date)
    book_history.append(info)

0 After We Collided
1 The Godfather
2 Harry Potter and the Sorcerer's Stone
3 Little Women
4 Little Women
5 Little Women
6 Little Women
7 Ready Player One
8 The Invisible Man
9 The Invisible Man
10 Jojo Rabbit
11 Forrest Gump
12 The Dry
13 The Witches
14 The Witches
15 Doctor Sleep
16 The Prestige
17 The Girl with the Dragon Tattoo
18 Fight Club
19 Harry Potter and the Goblet of Fire
20 No Country for Old Men
21 Murder on the Orient Express
22 Murder on the Orient Express
23 O Brother, Where Art Thou?
24 Gone Girl
25 The Hunger Games
26 Twilight
27 The Silence of the Lambs
28 The Green Mile
29 The Princess Bride
30 Rebecca
31 Rebecca
32 Harry Potter and the Half-Blood Prince
33 The War with Grandpa
34 Harry Potter and the Prisoner of Azkaban
35 Nocturnal Animals
36 Red Sparrow
37 The Outsiders
38 Shutter Island
39 The Help
40 Harry Potter and the Order of the Phoenix
41 Jurassic Park
42 Requiem for a Dream
43 Fifty Shades of Grey
44 Once Upon a Time in America
45 It
46 The Shining
47 D

KeyboardInterrupt: 

In [161]:
len(book_history)

1352

In [166]:
import pickle
# Save the list to pickle
with open('../dump/book_history_list','wb') as f:
    pickle.dump(book_history,f)

In [177]:
book_history_df = pd.DataFrame(book_history)

In [179]:
# Save the data to pickle file
book_history_df.to_pickle('../dump/book_history_data')

In [17]:
book_history_df.head()

,title,release_date,title_search,search_fiction_book,book_popularity,author_search,search_fiction_author,author_popularity
0,After We Collided,2020-10-23,83300.0,82300000.0,0.00,44900000.0,208000000,0.2159
1,The Godfather,1972-03-24,164.0,1050.0,0.16,327.0,4680,0.0699
2,Harry Potter and the Sorcerer's Stone,2001-11-16,6170.0,44100.0,0.14,67800.0,4560000,0.0149
3,Little Women,2019-12-25,191000.0,82500000.0,0.00,2760000.0,183000000,0.0151
4,Little Women,1994-12-25,982.0,14900.0,0.07,1860.0,117000,0.0159


In [181]:
# Test load the pickled df
book_history_df = pd.read_pickle('../dump/book_history_data')

In [18]:
# Check book_popularity and author_popularity
book_popularity_df = book_history_df[['title','book_popularity','author_popularity']].dropna()
book_popularity_df[book_popularity_df.book_popularity == 0]

,title,book_popularity,author_popularity
0,After We Collided,0.0,0.2159
3,Little Women,0.0,0.0151
7,Ready Player One,0.0,0.0012
8,The Invisible Man,0.0,0.1500
10,Jojo Rabbit,0.0,0.0064
...,...,...,...
1341,Island of the Blue Dolphins,0.0,0.0059
1345,Pretty Poison,0.0,0.0027
1346,The Wayward Bus,0.0,0.0176
1347,From the Terrace,0.0,0.0418


### Improve book popularity
(Include genre in general search: added book_google2() in fictiondb.py)

In [27]:
import pandas as pd
# Since 'budget' is the bottle neck in dropna process, use that to narrow down the search list
all_df = pd.read_pickle('../dump/all_data').dropna(subset=['budget'])

In [32]:
all_df['date'] = all_df.release_date.dt.strftime('%Y-%m-%d')
all_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1166 entries, 1 to 1572
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   movie_title          1166 non-null   object        
 1   rating               1166 non-null   float64       
 2   vote                 1166 non-null   int64         
 3   certificate          1166 non-null   object        
 4   genre                1166 non-null   object        
 5   release_date         1166 non-null   datetime64[ns]
 6   metascore            475 non-null    float64       
 7   keywords             1166 non-null   object        
 8   budget               1166 non-null   float64       
 9   opening_weekend_usa  849 non-null    float64       
 10  gross_usa            902 non-null    float64       
 11  gross_world          968 non-null    float64       
 12  runtime              1166 non-null   float64       
 13  director             1166 non-nul

In [33]:
# Create list to search
title_list = list(all_df.movie_title)
author_list = list(all_df.author)
date_list = list(all_df.date)
genre_list = list(all_df.genre)
lng = len(title_list)

In [29]:
%run -i '../py/fictiondb.py'

In [39]:
book_history_2 = []

In [40]:
for i in range(lng):
    book = title_list[i]
    author = author_list[i]
    date = date_list[i]
    genre = genre_list[i]
    
    print(i,book)
    
    info = book_google2(book,author,date,genre)
    book_history_2.append(info)
book_history_2

0 The Godfather
1 Harry Potter and the Sorcerer's Stone
2 Harry Potter and the Sorcerer's Stone
3 Little Women
4 Little Women
5 Little Women
6 Little Women
7 Little Women
8 Little Women
9 Little Women
10 Little Women
11 Little Women
12 Little Women
13 Little Women
14 Little Women
15 Ready Player One
16 The Invisible Man
17 The Invisible Man
18 Jojo Rabbit
19 Forrest Gump
20 Doctor Sleep
21 The Prestige
22 The Girl with the Dragon Tattoo
23 Fight Club
24 Harry Potter and the Goblet of Fire
25 No Country for Old Men
26 Murder on the Orient Express
27 Murder on the Orient Express
28 Murder on the Orient Express
29 Murder on the Orient Express
30 O Brother, Where Art Thou?
31 Gone Girl
32 The Hunger Games
33 The Hunger Games
34 Twilight
35 The Silence of the Lambs
36 The Silence of the Lambs
37 The Green Mile
38 The Princess Bride
39 Rebecca
40 Rebecca
41 Harry Potter and the Half-Blood Prince
42 Harry Potter and the Prisoner of Azkaban
43 Nocturnal Animals
44 Red Sparrow
45 The Outsiders


387 The Secret Garden
388 The Secret Garden
389 The Secret Garden
390 Road to Perdition
391 Road to Perdition
392 Fear and Loathing in Las Vegas
393 Fear and Loathing in Las Vegas
394 Carlito's Way
395 Carlito's Way
396 The Virgin Suicides
397 The Virgin Suicides
398 The Maltese Falcon
399 The Maltese Falcon
400 The Maltese Falcon
401 The Maltese Falcon
402 The Graduate
403 The Graduate
404 The Mist
405 The Mist
406 The Mountain Between Us
407 The Mountain Between Us
408 Aquamarine
409 Aquamarine
410 A Walk Among the Tombstones
411 A Walk Among the Tombstones
412 Chocolat
413 Chocolat
414 The Hunt for Red October
415 The Hunt for Red October
416 The November Man
417 The November Man
418 The Thin Red Line
419 The Thin Red Line
420 Body of Lies
421 Body of Lies
422 Pride and Prejudice and Zombies
423 Pride and Prejudice and Zombies
424 The Other Boleyn Girl
425 The Other Boleyn Girl
426 The Other Boleyn Girl
427 The Other Boleyn Girl
428 The Reader
429 The Reader
430 The 5th Wave
431 The

765 The Big Sleep
766 The Big Sleep
767 The Big Sleep
768 Tuck Everlasting
769 The Honey Pot
770 White Oleander
771 All Quiet on the Western Front
772 The Flight of the Phoenix
773 The Flight of the Phoenix
774 The Painted Veil
775 Pay It Forward
776 The Boys from Brazil
777 Valley of the Dolls
778 The World According to Garp
779 Mr. Popper's Penguins
780 Every Day
781 The Sword in the Stone
782 The Sword in the Stone
783 The Human Stain
784 What's Your Number?
785 Confessions of a Teenage Drama Queen
786 The Year of Living Dangerously
787 Toy Soldiers
788 The Ox-Bow Incident
789 The Natural
790 The Grapes of Wrath
791 Billy Lynn's Long Halftime Walk
792 The Horse Whisperer
793 Diary of a Wimpy Kid: The Long Haul
794 Labor Day
795 The Cotton Club
796 Stormbreaker
797 The Sentinel
798 The Sentinel
799 The Sentinel
800 The Sentinel
801 Catch-22
802 Mermaids
803 Death Sentence
804 A Hologram for the King
805 Sphere
806 Vanity Fair
807 The Bonfire of the Vanities
808 About Schmidt
809 Exit

1135 Thieves Like Us
1136 Drums Along the Mohawk
1137 A Rage in Harlem
1138 The Cobweb
1139 The Lonely Lady
1140 The Teahouse of the August Moon
1141 The Devil at 4 O'Clock
1142 Way of a Gaucho
1143 Wise Blood
1144 Legends of Oz: Dorothy's Return
1145 Where the Sidewalk Ends
1146 A Walk in the Sun
1147 Tai-Pan
1148 Saint Jack
1149 A Farewell to Arms
1150 Losing Isaiah
1151 Cinderella Liberty
1152 Mother Night
1153 Never Die Alone
1154 Twilight's Last Gleaming
1155 Radio Free Albemuth
1156 Lord Love a Duck
1157 The Heart Is a Lonely Hunter
1158 The Golden Bowl
1159 Stars and Bars
1160 Monte Walsh
1161 Boy on a Dolphin
1162 The Good Mother
1163 Finders Keepers
1164 Pretty Poison
1165 Anne of Green Gables


[{'title': 'The Godfather',
  'release_date': '1972-03-24',
  'genre': ['Crime', 'Drama'],
  'title_search': 192,
  'search_fiction_book': 728,
  'author_search': 258,
  'search_fiction_author': 4550,
  'book_popularity': 0.26,
  'author_popularity': 0.0567},
 {'title': "Harry Potter and the Sorcerer's Stone",
  'release_date': '2001-11-16',
  'genre': ['Adventure', 'Family', 'Fantasy'],
  'title_search': 5720,
  'search_fiction_book': 31300,
  'author_search': 68600,
  'search_fiction_author': 5680000,
  'book_popularity': 0.18,
  'author_popularity': 0.0121},
 {'title': "Harry Potter and the Sorcerer's Stone",
  'release_date': '2001-11-16',
  'genre': ['Adventure', 'Family', 'Fantasy'],
  'title_search': 5720,
  'search_fiction_book': 31300,
  'author_search': 68600,
  'search_fiction_author': 5680000,
  'book_popularity': 0.18,
  'author_popularity': 0.0121},
 {'title': 'Little Women',
  'release_date': '2019-12-25',
  'genre': ['Drama', 'Romance'],
  'title_search': 214000,
  'sea

In [43]:
book_history_2_df = pd.DataFrame(book_history_2)

In [44]:
book_history_2_df.to_pickle('../data/book_history_2_data')